In [ ]:
from trackscopy_fluorescence import image_analysis_tools as im
from trackscopy_fluorescence import swim_mode_analysis_tools as sm
import os
import numpy as np
import tifffile as tf
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pickle

In [ ]:
#PARENT_DIRECTORY = '///'
#SUB_DIRECTORY = '///'

PICKLE_DIRECTORY = 'results_image_analysis'

#DIRECTORY = os.path.join(PARENT_DIRECTORY,SUB_DIRECTORY,PICKLE_DIRECTORY)

DIRECTORY = os.path.join('sample_data',PICKLE_DIRECTORY)

FILE_NAME = 'motility_bacteria'

ext = '_swim_mode_analysis.pickle'

full_filename = (FILE_NAME+ext)
full_file =  os.path.join(DIRECTORY,full_filename)

res_path = os.path.join(DIRECTORY,'analysis_result_collective')

if os.path.exists(res_path)!= True:
    
    os.mkdir(res_path)

# Accessing Properties:

The loaded pickle file 'analysis' (the user can change it freely to any name) is an object (class) containing the following properties:


$\color{red}{swimmers}$ :  a list of Objects, corresponding to the sorted data, each inhereting their corresponding dataframes

$\color{red}{path}$ :  path to the file


$\color{red}{parameters\_red\_channel}$ : parameters used for image analysis of red channel

$\color{red}{parameters\_green\_channel}$ : parameters used for image analysis of green channel

$\color{red}{parameters\_swim\_mode\_detection}$ : parameters used for swim mode detection



### Swimmer object

As mentioned in the cell above, $\color{red}{analysis.swimmers}$ corresponds to a list of Objects. The swimmer object consists of the following properties:

$\color{red}{dataframe}$ :   the dataframe inherited from the main analysis sorted by ID

$\color{red}{mean\_speed}$ :    a float corresponding to the average velocity of the swimmer in microns/sec 

$\color{red}{run\_time}$ :     an integer corresponding to the run-time of the swimmer in frames

$\color{red}{episodes}$ :    a list of Objects, which are subsets of the swimmer sorted by a specific feature (swim-mode, orientation or flow)

$\color{red}{sorted\_episodes\_by}$ :    a string corresponding to the feature mentioned above

And it contains the following function:

$\color{blue}{extract\_episodes( )}$     default input: sort_by = 'SWIM-MODE', can be changed into 'FLOW' or 'ORIENTATION'

#### Episode object

As mentioned in the cell above, $\color{red}{analysis.swimmer[i].episodes}$ corresponds to a list of Objects. The episode object consists of the following properties:

$\color{red}{dataframe}$ :   the dataframe inherited from the swimmer sorted by a specific feature (swim-mode, orientation or flow)

$\color{red}{mean\_speed}$ :    a float corresponding to the average velocity of the episode in microns/sec 

$\color{red}{run\_time}$ :     an integer corresponding to the run-time of the episode in frames

$\color{red}{sorted\_by}$ :    a string corresponding to the feature mentioned above by which it has been sorted from a given swimmer

$\color{red}{identity}$ :    a string corresponding to the value of the feature (e.g., type of swim-mode like 'pull')


In the cell below, an example is given, how to access the different properties of the pickle file.

In [ ]:
analysis = im.load(full_file)

print('FILE NAME: ',FILE_NAME.split('/')[-1].split('.')[0])

a = 1

print('NUMBER OF SWIMMERS: ',len(analysis.swimmers))

for swimmer in analysis.swimmers:

    print(f'swimmer{a}')

    for episode in swimmer.episodes:

        print(episode.identity)
        
    a += 1
    
    
#print(analysis.path)
print(analysis.parameters_red_channel)
#print(analysis.parameters_green_channel)
#print(analysis.parameters_swim_mode_detection)
#print(analysis.swimmers[0].episodes[0].dataframe)

## Example function to plot trajectories detected by swim-modes


If the user wants to save the plot from the $\color{blue}{plot\_tracks( )}$ function, the corresponding path of the image has to be specified as an input inside the function. The syntax would be $\color{blue}{plot\_tracks([path\_to\_image])}$  

In [ ]:
def plot_tracks(self,path=None):

    Alpha = np.linspace(0.5,1,len(self.swimmers))

    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,5))

    for i,swimmer in enumerate(self.swimmers):
        
        df = swimmer.dataframe

        X = np.array(df['C:X-COORD'],float)
        Y = np.array(df['C:Y-COORD'],float)
        modes = np.array(df['SWIM-MODE'],str)
        velocities = np.array(df['VELOCITY'],float)
        frames = np.array(df['FRAME'],int)

        pushers = np.where(modes=='push')[0]
        pullers = np.where(modes=='pull')[0]
        wrappers = np.where(modes == 'wrap')[0]
        passive = np.where(modes == 'passive')[0]


        if len(pushers) != 0:
            ax1.plot(X[pushers],Y[pushers],'b.',alpha=Alpha[i],markersize=8)
            ax1.text(np.mean(X[pushers])+5,np.mean(Y[pushers])+5,f'{i}')
            ax2.plot(frames[pushers],velocities[pushers],'b.',alpha=Alpha[i],markersize=8)
        if len(pullers) != 0:
            ax1.plot(X[pullers],Y[pullers],'r.',alpha=Alpha[i],markersize=8)
            ax1.text(np.mean(X[pullers])+5,np.mean(Y[pullers])+5,f'{i}')
            ax2.plot(frames[pullers],velocities[pullers],'r.',alpha=Alpha[i],markersize=8)
        if len(wrappers) != 0:
            ax1.plot(X[wrappers],Y[wrappers],'g.',alpha=Alpha[i],markersize=8)
            ax1.text(np.mean(X[wrappers])+5,np.mean(Y[wrappers])+5,f'{i}')
            ax2.plot(frames[wrappers],velocities[wrappers],'g.',alpha=Alpha[i],markersize=8)
        if len(passive) != 0:
            ax1.plot(X[passive],Y[passive],'k.',alpha=Alpha[i],markersize=8)
            ax1.text(np.mean(X[passive])+5,np.mean(Y[passive])+5,f'{i}')
            ax2.plot(frames[passive],velocities[passive],'k.',alpha=Alpha[i],markersize=8)


    ax1.set_xlabel(r'X-coordinate in $\mu m$',fontsize=12)
    ax1.set_ylabel(r'Y-coordinate in $\mu m$',fontsize=12)

    ax2.set_xlabel('Frames',fontsize=12)
    ax2.set_ylabel(r'Velocity in $\mu m/s$',fontsize=12)

    if path is None:
        pass
    else:
        plt.savefig(path,facecolor='white')


Blue corresponds to 'push', red corresponds to 'pull', green corresponds to 'wrap' and black corresponds to 'passive'.

In [ ]:
plot_tracks(analysis)

### Accessing properties of multiple files (example syntax)

In [ ]:
'''PARENT_DIRECTORY = '///'

res = 'analysis_result_collective'
cal_filename = 'calibration.txt'

SUB_DIRECTORIES = [x for x in next(os.walk(PARENT_DIRECTORY), (None, [], None))[1] if x!=res]

for SUB_DIRECTORY in SUB_DIRECTORIES:
    
    print(f'Sub-directory: {SUB_DIRECTORY}')
    
    DIRECTORY = os.path.join(PARENT_DIRECTORY,SUB_DIRECTORY)
    res_path = os.path.join(DIRECTORY,'results_image_analysis')
    
    FILE_NAMES = [os.path.join(res_path,x) for x in next(os.walk(res_path), (None, None, []))[2] if x!= cal_filename and if x.split('.')[-1]==ext]            
            
    for FILE_NAME in FILE_NAMES:
        
        print(FILE_NAME.split('/')[-1].split('.')[0])
        
        analysis = im.load(FILE_NAME)
        a = 0
        for swimmer in analysis.swimmers:

            print(f'swimmer{a}')
            
            for episode in swimmer.episodes:

                print(episode.identity)

            a += 1'''